In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
from rasterstats import zonal_stats

In [ ]:
import sys
# Adjust the path below to match the location of your QGIS Python modules
sys.path.append('C:/Program Files/QGIS 3.32.3/apps/qgis/python')
from qgis.core import *


In [ ]:
from qgis.core import QgsApplication

# Set the prefix path to the root of the QGIS installation
QgsApplication.setPrefixPath("C:/Program Files/QGIS 3.32.3", True)

qgs = QgsApplication([], False)
qgs.initQgis()

# Your PyQGIS code goes here

qgs.exitQgis()



In [ ]:
path = r"G:\My Drive\SIGNAL\data\eems_avocado_esha\rwmp_area.gpkg"

In [ ]:
# Load your layer from the GeoPackage
gdf = gpd.read_file(path)



In [ ]:
# Get the bounds of your study area
xmin, ymin, xmax, ymax = gdf.total_bounds

# Define the size of your fishnet cells
cell_size = 60

# Generate the coordinates for the fishnet
x_coords = np.arange(xmin, xmax + cell_size, cell_size)
y_coords = np.arange(ymin, ymax + cell_size, cell_size)

# Create the fishnet grid cells
grid_cells = []
for x in x_coords:
    for y in y_coords:
        # Define the polygon for each grid cell
        polygon = Polygon([(x, y), (x + cell_size, y), (x + cell_size, y - cell_size), (x, y - cell_size)])
        grid_cells.append(polygon)

# Create a GeoDataFrame from the grid cells
grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'])

# Set the same CRS as your input layer
grid.crs = gdf.crs

# Save the grid to a new file (optional)
grid.to_file('G:/My Drive/SIGNAL/data/eems_avocado_esha/fishnet_grid.gpkg', layer='fishnet', driver='GPKG')

In [ ]:
# Ensure both GeoDataFrames are in the same CRS
grid = grid.to_crs(gdf.crs)

# Perform the spatial intersection (overlay) to crop the grid
cropped_grid = gpd.overlay(grid, gdf, how='intersection')

# Save the cropped grid to a new file (optional)
cropped_grid.to_file('G:/My Drive/SIGNAL/data/eems_avocado_esha/cropped_fishnet_grid.gpkg', layer='cropped_fishnet', driver='GPKG')

In [ ]:
# Load the fishnet grid

cropped_grid = gpd.read_file(r'G:/My Drive/SIGNAL/data/eems_avocado_esha/cropped_fishnet_grid.gpkg')

In [ ]:

# Load the ag_greenbelt shapes
ag_greenbelt = gpd.read_file("G:/My Drive/SIGNAL/data/zip_databasin/ag_greenbelts_02182021/ag_greenbelts_02182021.gpkg", driver='GPKG')

In [ ]:
# Perform a spatial join
join = gpd.sjoin(cropped_grid, ag_greenbelt, how="left", op="intersects")


In [ ]:
join

In [ ]:
# Initialize 'greenbelt_presence' in the original fishnet DataFrame
cropped_grid['greenbelt_presence'] = 0

In [ ]:
# Update 'greenbelt_presence' based on 'index_right' from the join
# If 'index_right' is not NaN, set 'greenbelt_presence' to 1
cropped_grid['greenbelt_presence'] = pd.notnull(join['index_right']).astype(int)

In [ ]:
# Reset the index of the cropped_grid DataFrame
cropped_grid = cropped_grid.reset_index(drop=True)

# Remove duplicates in the fishnet DataFrame and keep the first occurrence
fishnet = cropped_grid[~cropped_grid.index.duplicated(keep='first')]

In [ ]:
# Save the updated fishnet grid
cropped_grid.to_file('updated_fishnet_with_greenbelt_presence.gpkg')


In [ ]:


# Assuming 'cropped_grid' is your clipped fishnet grid GeoDataFrame
# And 'raster_file.tif' is the path to your raster file

# Define the statistics you want to extract from the raster for each polygon
# In this case, we'll use 'mean' but you can use 'median', 'majority', 'sum', etc.
stats = zonal_stats(cropped_grid, 'G:/My Drive/SIGNAL/data/eems_avocado_esha/rasters/ag_adjacency.tif', stats='mean', all_touched=True)

# Extract the 'mean' value from the stats and assign it to a new column in your GeoDataFrame
cropped_grid['RasterValue'] = [stat['mean'] for stat in stats]

# Now, 'cropped_grid' contains a new column 'RasterValue' with the mean raster values for each cell


In [ ]:
cropped_grid

In [ ]:
cropped_grid['RasterValue'].sum()
